In [1]:
import pickle
import cobra
import lftc
import igraph
import pandas as pd
import re

In [2]:
results = pickle.load(open('test_data/results_e_coli.p', 'rb'))[39][0]
len(results)

141

In [3]:
with open('test_data/REACTIONSwt5h.txt') as f:
    allLines = f.readlines()
reactionLines = list(filter(lambda l: re.match('^\w', l), allLines))
coreReactionNames = set([re.sub('\s.*$', '', l) for l in reactionLines])
coreReactionNames = set([re.sub('[\(,\)]', '_', l) for l in coreReactionNames])

print(str(len(coreReactionNames)) + ' core reactions in transitions file')

127 core reactions in transitions file


In [4]:
model = cobra.io.read_sbml_model('test_data/EciJR904TKs_sbml3.xml')
feed='EX_glc_e_'

In [5]:
len(results)/len(model.reactions)

0.1309192200557103

In [6]:
sortedMetabolites = pd.DataFrame([(m.id, len(m.reactions)) for m in model.metabolites]).sort_values(1,ascending=False)
set(sortedMetabolites[0:0][0]).difference(lftc.currencyMetabolites)

set()

In [7]:
len(model.reactions)

1077

In [8]:
modelGraph = igraph.Graph(directed=False)
currencyMetabolites=lftc.currencyMetabolites
modelGraph.add_vertex(name=feed)
newCoreReactionNames = set()
unexploredReactionNames = {feed} # this starts with just the feed name
while len(unexploredReactionNames) > 0:
    reactionName = unexploredReactionNames.pop()
    newCoreReactionNames.update([reactionName])
    # modelGraph.add_vertex(name=reactionName)
    reaction = model.reactions.get_by_id(reactionName)
    connectedMetabolites = {m.id for m in reaction.reactants}
    connectedMetabolites.update({m.id for m in reaction.products})
    connectedMetabolites = connectedMetabolites.difference(currencyMetabolites)
    sourceNode = modelGraph.vs["name"].index(reactionName)
    
    for metaboliteName in connectedMetabolites:
        metabolite = model.metabolites.get_by_id(metaboliteName)
        connectedReactionNames = {r.id for r in metabolite.reactions}
        # connectedReactionNames = connectedReactionNames.intersection(coreReactionNames)
        newReactionsThisLoop = connectedReactionNames.difference(newCoreReactionNames)
        unexploredReactionNames.update(newReactionsThisLoop)
        newCoreReactionNames.update(newReactionsThisLoop)
        for newReactionName in connectedReactionNames:
            # print("connected to", reactionName, len(newReactionName))
            if not newReactionName in modelGraph.vs["name"]:
                modelGraph.add_vertex(name=newReactionName)
            
            targetNode = modelGraph.vs["name"].index(newReactionName)
            if not modelGraph.are_connected(sourceNode, targetNode):
                modelGraph.add_edge(source=sourceNode, target=targetNode)

In [9]:
print(modelGraph)

IGRAPH UN-- 1028 8212 --
+ attr: name (v)
+ edges (vertex names):
     EX_glc_e_ -- EX_glc_e_, EX_glc_e_, TREHe, GLCDe, GLCt2, GLCpts
         TREHe -- EX_glc_e_, TREHe, TREHe, GLCDe, GLCt2, GLCpts, EX_tre_e_,
TREpts, EX_h2o_e_, DMSOR2e, H2Ot, TMAOR2e, NADDPe, DMSOR1e, TMAOR1e
         GLCDe -- EX_glc_e_, TREHe, GLCDe, GLCDe, GLCt2, GLCpts, EX_h2o_e_,
DMSOR2e, H2Ot, TMAOR2e, NADDPe, DMSOR1e, TMAOR1e, GLCNt2r, EX_glcn_e_
         GLCt2 -- EX_glc_e_, TREHe, GLCDe, GLCt2, GLCt2, GLCpts, TRE6PH, HEX1,
MLTG5, MLTG3, GALS3, AMALT3, XYLI2i, AMALT1, MLTG4, AMALT4, TREH, LACZ, MLTG1,
AMALT2, MLTG2, G1PP
        GLCpts -- EX_glc_e_, TREHe, GLCDe, GLCt2, GLCpts, GLCpts, TREpts,
SUCpts, MALTpts, PPCK, UAGCVT, FRUpts2, PYK, ACGApts, SBTpts, KDOPS, PPC,
PSCVT, DDPA, DHAPT, GAMpts, MANpts, FRUpts, MNLpts, PPS, GALTpts, ACMANApts,
ENO, TRE6PH, TRE6PS, ALATA_L2, PDH, VPAMT, ACNML, ACHBS, DHDPS, GLCRAL,
ICHORT, TRPAS2, ME1, ALATA_D2, SERD_D, SHCHCS2, ACLS, ANS, SERD_L, EDA,
ALATA_L, L_LACD2, CHRPL, HOPN

In [10]:
modelGraph.is_connected(mode=igraph.WEAK)

True

In [11]:
for reactionName in results:
    modelGraph.vs[modelGraph.vs["name"].index(reactionName)]['class'] = 'newcore'

for reactionName in coreReactionNames:
    modelGraph.vs[modelGraph.vs["name"].index(reactionName)]['class'] = 'oldcore'
    


In [12]:
modelGraph.save('test_data/graph.graphml', 'graphml')